In [1]:
import argparse
import logging
import math
import os
import random
import time
from copy import deepcopy
from pathlib import Path
from threading import Thread

import numpy as np
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

import test  # import test.py to get mAP after each epoch
from models.experimental import attempt_load
from models.yolo import Model
from utils.autoanchor import check_anchors
from utils.datasets import create_dataloader
from utils.general import labels_to_class_weights, increment_path, labels_to_image_weights, init_seeds, \
    fitness, strip_optimizer, get_latest_run, check_dataset, check_file, check_git_status, check_img_size, \
    check_requirements, print_mutation, set_logging, one_cycle, colorstr
from utils.google_utils import attempt_download
from utils.loss import ComputeLoss, ComputeLossAuxOTA
from utils.plots import plot_images, plot_labels, plot_results, plot_evolution
from utils.torch_utils import ModelEMA, select_device, intersect_dicts, torch_distributed_zero_first, is_parallel
from utils.wandb_logging.wandb_utils import WandbLogger, check_wandb_resume

logger = logging.getLogger(__name__)

from huggingface_hub import HfApi

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--weights', type=str, default='', help='initial weights path')
parser.add_argument('--cfg', type=str, default='cfg/training/image_tagging_training.yaml', help='model.yaml path')
parser.add_argument('--data', type=str, default='data/image_tagging_data.yaml', help='data.yaml path')
parser.add_argument('--hyp', type=str, default='data/hyp.scratch.custom.yaml', help='hyperparameters path')
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--batch-size', type=int, default=1, help='total batch size for all GPUs')
parser.add_argument('--img-size', nargs='+', type=int, default=[1280, 1280], help='[train, test] image sizes')
parser.add_argument('--rect', action='store_true', help='rectangular training')
parser.add_argument('--resume', nargs='?', const=True, default=False, help='resume most recent training')
parser.add_argument('--nosave', action='store_true', help='only save final checkpoint')
parser.add_argument('--notest', action='store_true', help='only test final epoch')
parser.add_argument('--noautoanchor', action='store_true', help='disable autoanchor check')
parser.add_argument('--evolve', action='store_true', help='evolve hyperparameters')
parser.add_argument('--bucket', type=str, default='', help='gsutil bucket')
parser.add_argument('--cache-images', action='store_true', help='cache images for faster training')
parser.add_argument('--image-weights', action='store_true', help='use weighted image selection for training')
parser.add_argument('--device', default='0', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
parser.add_argument('--multi-scale', action='store_true', help='vary img-size +/- 50%%')
parser.add_argument('--single-cls', action='store_true', help='train multi-class data as single-class')
parser.add_argument('--adam', action='store_true', help='use torch.optim.Adam() optimizer')
parser.add_argument('--sync-bn', action='store_true', help='use SyncBatchNorm, only available in DDP mode')
parser.add_argument('--local_rank', type=int, default=-1, help='DDP parameter, do not modify')
parser.add_argument('--workers', type=int, default=8, help='maximum number of dataloader workers')
parser.add_argument('--project', default='runs/train', help='save to project/name')
parser.add_argument('--entity', default=None, help='W&B entity')
parser.add_argument('--name', default='exp', help='save to project/name')
parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
parser.add_argument('--quad', action='store_true', help='quad dataloader')
parser.add_argument('--linear-lr', action='store_true', help='linear LR')
parser.add_argument('--label-smoothing', type=float, default=0.0, help='Label smoothing epsilon')
parser.add_argument('--upload_dataset', action='store_true', help='Upload dataset as W&B artifact table')
parser.add_argument('--bbox_interval', type=int, default=-1, help='Set bounding-box image logging interval for W&B')
parser.add_argument('--save_period', type=int, default=-1, help='Log model after every "save_period" epoch')
parser.add_argument('--artifact_alias', type=str, default="latest", help='version of dataset artifact to be used')
parser.add_argument('--v5-metric', action='store_true', help='assume maximum recall as 1.0 in AP calculation')

# to fix bug
parser.add_argument(
            "-i", "--ip", help="a dummy argument to fool ipython", default="1")
parser.add_argument(
    "-s", "--stdin", help="a dummy argument to fool ipython", default="1")
parser.add_argument(
    "-c", "--control", help="a dummy argument to fool ipython", default="1")
parser.add_argument(
    "-b", "--hb", help="a dummy argument to fool ipython", default="1")
parser.add_argument(
    "-K", "--Session.key", help="a dummy argument to fool ipython", default="1")
parser.add_argument(
    "-S", "--Session.signature_scheme", help="a dummy argument to fool ipython", default="1")
parser.add_argument(
    "-l", "--shell", help="a dummy argument to fool ipython", default="1")
parser.add_argument(
    "-t", "--transport", help="a dummy argument to fool ipython", default="1")
parser.add_argument(
    "-o", "--iopub", help="a dummy argument to fool ipython", default="1")
parser.add_argument('--f', default = 'runtime_ipynb.json')

opt = parser.parse_args()

# Set DDP variables
opt.world_size = int(os.environ['WORLD_SIZE']) if 'WORLD_SIZE' in os.environ else 1
opt.global_rank = int(os.environ['RANK']) if 'RANK' in os.environ else -1
set_logging(opt.global_rank)
#if opt.global_rank in [-1, 0]:
#    check_git_status()
#    check_requirements()

opt.exist_ok = True

In [3]:
wandb_run = check_wandb_resume(opt)
if opt.resume and not wandb_run:  # resume an interrupted run
    ckpt = opt.resume if isinstance(opt.resume, str) else get_latest_run()  # specified or most recent path
    assert os.path.isfile(ckpt), 'ERROR: --resume checkpoint does not exist'
    apriori = opt.global_rank, opt.local_rank
    with open(Path(ckpt).parent.parent / 'opt.yaml') as f:
        opt = argparse.Namespace(**yaml.load(f, Loader=yaml.SafeLoader))  # replace
    opt.cfg, opt.weights, opt.resume, opt.batch_size, opt.global_rank, opt.local_rank = '', ckpt, True, opt.total_batch_size, *apriori  # reinstate
    logger.info('Resuming training from %s' % ckpt)
else:
    # opt.hyp = opt.hyp or ('hyp.finetune.yaml' if opt.weights else 'hyp.scratch.yaml')
    opt.data, opt.cfg, opt.hyp = check_file(opt.data), check_file(opt.cfg), check_file(opt.hyp)  # check files
    assert len(opt.cfg) or len(opt.weights), 'either --cfg or --weights must be specified'
    opt.img_size.extend([opt.img_size[-1]] * (2 - len(opt.img_size)))  # extend to 2 sizes (train, test)
    opt.name = 'evolve' if opt.evolve else opt.name
    opt.save_dir = increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok | opt.evolve)  # increment run

In [4]:
# DDP mode
opt.total_batch_size = opt.batch_size
device = select_device(opt.device, batch_size=opt.batch_size)
if opt.local_rank != -1:
    assert torch.cuda.device_count() > opt.local_rank
    torch.cuda.set_device(opt.local_rank)
    device = torch.device('cuda', opt.local_rank)
    dist.init_process_group(backend='nccl', init_method='env://')  # distributed backend
    assert opt.batch_size % opt.world_size == 0, '--batch-size must be multiple of CUDA device count'
    opt.batch_size = opt.total_batch_size // opt.world_size


YOLOR 🚀 2023-5-8 torch 1.10.0+cu111 CUDA:0 (NVIDIA GeForce GTX 1650, 3912.0MB)



In [5]:
# Hyperparameters
with open(opt.hyp) as f:
    hyp = yaml.load(f, Loader=yaml.SafeLoader)  # load hyps

In [6]:
# Train
logger.info(opt)
if not opt.evolve:
    tb_writer = None  # init loggers
    if opt.global_rank in [-1, 0]:
        prefix = colorstr('tensorboard: ')
        logger.info(f"{prefix}Start with 'tensorboard --logdir {opt.project}', view at http://localhost:6006/")
        tb_writer = SummaryWriter(opt.save_dir)  # Tensorboard
    # train(hyp, opt, device, tb_writer)

Namespace(weights='', cfg='cfg/training/image_tagging_training.yaml', data='data/image_tagging_data.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=10, batch_size=1, img_size=[1280, 1280], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='exp', exist_ok=True, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', v5_metric=False, ip='127.0.0.1', stdin='9013', control='9011', hb='9010', shell='9012', transport='"tcp"', iopub='9014', f='/home/tavandai/.local/share/jupyter/runtime/kernel-v2-4526wWn1YE5jbFNi.json', world_size=1, global_rank=-1, save_dir='runs/train/exp', total_batch_size=1, **{'Session.key': 'b"1de07427-23ef-44c6-8ba0-5669f4d5cf17"', 'Session.signature_scheme': '"hmac

In [7]:
api = HfApi()

logger.info(colorstr('hyperparameters: ') + ', '.join(f'{k}={v}' for k, v in hyp.items()))
save_dir, epochs, batch_size, total_batch_size, weights, rank = \
    Path(opt.save_dir), opt.epochs, opt.batch_size, opt.total_batch_size, opt.weights, opt.global_rank


hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.2, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0, paste_in=0.0, loss_ota=1


In [8]:
# Directories
wdir = save_dir / 'weights'
wdir.mkdir(parents=True, exist_ok=True)  # make dir
last = wdir / 'last.pt'
best = wdir / 'best.pt'
results_file = save_dir / 'results.txt'

In [9]:
with open(save_dir / 'hyp.yaml', 'w') as f:
    yaml.dump(hyp, f, sort_keys=False)
with open(save_dir / 'opt.yaml', 'w') as f:
    yaml.dump(vars(opt), f, sort_keys=False)


In [10]:
# Configure
plots = not opt.evolve  # create plots
cuda = device.type != 'cpu'
init_seeds(2 + rank)
with open(opt.data) as f:
    data_dict = yaml.load(f, Loader=yaml.SafeLoader)  # data dict
is_coco = opt.data.endswith('coco.yaml')

# Logging- Doing this before checking the dataset. Might update data_dict
loggers = {'wandb': None}  # loggers dict
if rank in [-1, 0]:
    opt.hyp = hyp  # add hyperparameters
    run_id = torch.load(weights).get('wandb_id') if weights.endswith('.pt') and os.path.isfile(weights) else None
    wandb_logger = WandbLogger(opt, Path(opt.save_dir).stem, run_id, data_dict)
    loggers['wandb'] = wandb_logger.wandb
    data_dict = wandb_logger.data_dict
    if wandb_logger.wandb:
        weights, epochs, hyp = opt.weights, opt.epochs, opt.hyp  # WandbLogger might update weights, epochs if resuming

nc = 1 if opt.single_cls else int(data_dict['nc'])  # number of classes
names = ['item'] if opt.single_cls and len(data_dict['names']) != 1 else data_dict['names']  # class names
assert len(names) == nc, '%g names found for nc=%g dataset in %s' % (len(names), nc, opt.data)  # check


wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)


In [11]:
pretrained = weights.endswith('.pt')
if pretrained:
    with torch_distributed_zero_first(rank):
        attempt_download(weights)  # download if not found locally
    ckpt = torch.load(weights, map_location=device)  # load checkpoint
    model = Model(opt.cfg or ckpt['model'].yaml, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create
    exclude = ['anchor'] if (opt.cfg or hyp.get('anchors')) and not opt.resume else []  # exclude keys
    state_dict = ckpt['model'].float().state_dict()  # to FP32
    state_dict = intersect_dicts(state_dict, model.state_dict(), exclude=exclude)  # intersect
    model.load_state_dict(state_dict, strict=False)  # load
    logger.info('Transferred %g/%g items from %s' % (len(state_dict), len(model.state_dict()), weights))  # report
else:
    model = Model(opt.cfg, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create


                 from  n    params  module                                  arguments                     
  0                -1  1         0  models.common.ReOrg                     []                            
  1                -1  1      7040  models.common.Conv                      [12, 64, 3, 1]                
  2                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  3                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  4                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

In [12]:
with torch_distributed_zero_first(rank):
    check_dataset(data_dict)  # check
train_path = data_dict['train']
test_path = data_dict['val']

In [13]:
# Freeze
freeze = []  # parameter names to freeze (full or partial)
for k, v in model.named_parameters():
    v.requires_grad = True  # train all layers
    if any(x in k for x in freeze):
        print('freezing %s' % k)
        v.requires_grad = False


In [14]:
# Optimizer
nbs = 64  # nominal batch size
accumulate = max(round(nbs / total_batch_size), 1)  # accumulate loss before optimizing
hyp['weight_decay'] *= total_batch_size * accumulate / nbs  # scale weight_decay
logger.info(f"Scaled weight_decay = {hyp['weight_decay']}")

pg0, pg1, pg2 = [], [], []  # optimizer parameter groups
for k, v in model.named_modules():
    if hasattr(v, 'bias') and isinstance(v.bias, nn.Parameter):
        pg2.append(v.bias)  # biases
    if isinstance(v, nn.BatchNorm2d):
        pg0.append(v.weight)  # no decay
    elif hasattr(v, 'weight') and isinstance(v.weight, nn.Parameter):
        pg1.append(v.weight)  # apply decay
    if hasattr(v, 'im'):
        if hasattr(v.im, 'implicit'):           
            pg0.append(v.im.implicit)
        else:
            for iv in v.im:
                pg0.append(iv.implicit)
    if hasattr(v, 'imc'):
        if hasattr(v.imc, 'implicit'):           
            pg0.append(v.imc.implicit)
        else:
            for iv in v.imc:
                pg0.append(iv.implicit)
    if hasattr(v, 'imb'):
        if hasattr(v.imb, 'implicit'):           
            pg0.append(v.imb.implicit)
        else:
            for iv in v.imb:
                pg0.append(iv.implicit)
    if hasattr(v, 'imo'):
        if hasattr(v.imo, 'implicit'):           
            pg0.append(v.imo.implicit)
        else:
            for iv in v.imo:
                pg0.append(iv.implicit)
    if hasattr(v, 'ia'):
        if hasattr(v.ia, 'implicit'):           
            pg0.append(v.ia.implicit)
        else:
            for iv in v.ia:
                pg0.append(iv.implicit)
    if hasattr(v, 'attn'):
        if hasattr(v.attn, 'logit_scale'):   
            pg0.append(v.attn.logit_scale)
        if hasattr(v.attn, 'q_bias'):   
            pg0.append(v.attn.q_bias)
        if hasattr(v.attn, 'v_bias'):  
            pg0.append(v.attn.v_bias)
        if hasattr(v.attn, 'relative_position_bias_table'):  
            pg0.append(v.attn.relative_position_bias_table)
    if hasattr(v, 'rbr_dense'):
        if hasattr(v.rbr_dense, 'weight_rbr_origin'):  
            pg0.append(v.rbr_dense.weight_rbr_origin)
        if hasattr(v.rbr_dense, 'weight_rbr_avg_conv'): 
            pg0.append(v.rbr_dense.weight_rbr_avg_conv)
        if hasattr(v.rbr_dense, 'weight_rbr_pfir_conv'):  
            pg0.append(v.rbr_dense.weight_rbr_pfir_conv)
        if hasattr(v.rbr_dense, 'weight_rbr_1x1_kxk_idconv1'): 
            pg0.append(v.rbr_dense.weight_rbr_1x1_kxk_idconv1)
        if hasattr(v.rbr_dense, 'weight_rbr_1x1_kxk_conv2'):   
            pg0.append(v.rbr_dense.weight_rbr_1x1_kxk_conv2)
        if hasattr(v.rbr_dense, 'weight_rbr_gconv_dw'):   
            pg0.append(v.rbr_dense.weight_rbr_gconv_dw)
        if hasattr(v.rbr_dense, 'weight_rbr_gconv_pw'):   
            pg0.append(v.rbr_dense.weight_rbr_gconv_pw)
        if hasattr(v.rbr_dense, 'vector'):   
            pg0.append(v.rbr_dense.vector)

if opt.adam:
    optimizer = optim.Adam(pg0, lr=hyp['lr0'], betas=(hyp['momentum'], 0.999))  # adjust beta1 to momentum
else:
    optimizer = optim.SGD(pg0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)

optimizer.add_param_group({'params': pg1, 'weight_decay': hyp['weight_decay']})  # add pg1 with weight_decay
optimizer.add_param_group({'params': pg2})  # add pg2 (biases)
logger.info('Optimizer groups: %g .bias, %g conv.weight, %g other' % (len(pg2), len(pg1), len(pg0)))
del pg0, pg1, pg2

# Scheduler https://arxiv.org/pdf/1812.01187.pdf
# https://pytorch.org/docs/stable/_modules/torch/optim/lr_scheduler.html#OneCycleLR
if opt.linear_lr:
    lf = lambda x: (1 - x / (epochs - 1)) * (1.0 - hyp['lrf']) + hyp['lrf']  # linear
else:
    lf = one_cycle(1, hyp['lrf'], epochs)  # cosine 1->hyp['lrf']
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)
# plot_lr_scheduler(optimizer, scheduler, epochs)

Scaled weight_decay = 0.0005
Optimizer groups: 115 .bias, 115 conv.weight, 115 other


In [15]:
# EMA
ema = ModelEMA(model) if rank in [-1, 0] else None

# Resume
start_epoch, best_fitness = 0, 0.0
if pretrained:
    # Optimizer
    if ckpt['optimizer'] is not None:
        optimizer.load_state_dict(ckpt['optimizer'])
        best_fitness = ckpt['best_fitness']

    # EMA
    if ema and ckpt.get('ema'):
        ema.ema.load_state_dict(ckpt['ema'].float().state_dict())
        ema.updates = ckpt['updates']

    # Results
    if ckpt.get('training_results') is not None:
        results_file.write_text(ckpt['training_results'])  # write results.txt

    # Epochs
    start_epoch = ckpt['epoch'] + 1
    if opt.resume:
        assert start_epoch > 0, '%s training to %g epochs is finished, nothing to resume.' % (weights, epochs)
    if epochs < start_epoch:
        logger.info('%s has been trained for %g epochs. Fine-tuning for %g additional epochs.' %
                    (weights, ckpt['epoch'], epochs))
        epochs += ckpt['epoch']  # finetune additional epochs

    del ckpt, state_dict

In [16]:
# Image sizes
gs = max(int(model.stride.max()), 32)  # grid size (max stride)
nl = model.model[-1].nl  # number of detection layers (used for scaling hyp['obj'])
imgsz, imgsz_test = [check_img_size(x, gs) for x in opt.img_size]  # verify imgsz are gs-multiples

In [17]:
# DP mode
if cuda and rank == -1 and torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)

# SyncBatchNorm
if opt.sync_bn and cuda and rank != -1:
    model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model).to(device)
    logger.info('Using SyncBatchNorm()')


In [18]:
# Trainloader
dataloader, dataset = create_dataloader(train_path, imgsz, batch_size, gs, opt,
                                        hyp=hyp, augment=False, cache=opt.cache_images, rect=opt.rect, rank=rank,
                                        world_size=opt.world_size, workers=opt.workers,
                                        image_weights=opt.image_weights, quad=opt.quad, prefix=colorstr('train: '))
mlc = np.concatenate(dataset.labels, 0).shape[1] -5  # max label class
nb = len(dataloader)  # number of batches
assert mlc < nc + 1, 'Label class %g exceeds nc=%g in %s. Possible class labels are 0-%g' % (mlc, nc, opt.data, nc - 1)

train: Scanning 'custom_data_multilabels/labels/train.cache' images and labels... 36498 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 36498/36498 [00:00<?, ?it/s]


In [19]:
if rank in [-1, 0]:
    testloader = create_dataloader(test_path, imgsz_test, batch_size * 2, gs, opt,  # testloader
                                    hyp=hyp, cache=opt.cache_images and not opt.notest, rect=True, rank=-1,
                                    world_size=opt.world_size, workers=opt.workers,
                                    pad=0.5, prefix=colorstr('val: '), 
                                    nc = nc)[0]

    if not opt.resume:
        labels = np.concatenate(dataset.labels, 0)
        c = np.where(labels[:,5:] == 1)[1]  # classes
        # cf = torch.bincount(c.long(), minlength=nc) + 1.  # frequency
        # model._initialize_biases(cf.to(device))
        if plots:
            #plot_labels(labels, names, save_dir, loggers)
            if tb_writer:
                tb_writer.add_histogram('classes', c, 0)

        # Anchors
        if not opt.noautoanchor:
            check_anchors(dataset, model=model, thr=hyp['anchor_t'], imgsz=imgsz)
        model.half().float()  # pre-reduce anchor precision


val: Scanning 'custom_data_multilabels/labels/val.cache' images and labels... 4562 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 4562/4562 [00:00<?, ?it/s]



autoanchor: Analyzing anchors... anchors/target = 5.65, Best Possible Recall (BPR) = 0.9820


In [20]:
if cuda and rank != -1:
    model = DDP(model, device_ids=[opt.local_rank], output_device=opt.local_rank,
                # nn.MultiheadAttention incompatibility with DDP https://github.com/pytorch/pytorch/issues/26698
                find_unused_parameters=any(isinstance(layer, nn.MultiheadAttention) for layer in model.modules()))


In [21]:
# Model parameters
hyp['box'] *= 3. / nl  # scale to layers
hyp['cls'] *= nc / 80. * 3. / nl  # scale to classes and layers
hyp['obj'] *= (imgsz / 640) ** 2 * 3. / nl  # scale to image size and layers
hyp['label_smoothing'] = opt.label_smoothing
model.nc = nc  # attach number of classes to model
model.hyp = hyp  # attach hyperparameters to model
model.gr = 1.0  # iou loss ratio (obj_loss = 1.0 or iou)
model.class_weights = labels_to_class_weights(dataset.labels, nc).to(device) * nc  # attach class weights
model.names = names

In [22]:
# Start training
t0 = time.time()
nw = max(round(hyp['warmup_epochs'] * nb), 1000)  # number of warmup iterations, max(3 epochs, 1k iterations)
# nw = min(nw, (epochs - start_epoch) / 2 * nb)  # limit warmup to < 1/2 of training
maps = np.zeros(nc)  # mAP per class
results = (0, 0, 0, 0, 0, 0, 0)  # P, R, mAP@.5, mAP@.5-.95, val_loss(box, obj, cls)
scheduler.last_epoch = start_epoch - 1  # do not move
scaler = amp.GradScaler(enabled=cuda)
compute_loss_ota = ComputeLossAuxOTA(model)  # init loss class
compute_loss = ComputeLoss(model)  # init loss class
logger.info(f'Image sizes {imgsz} train, {imgsz_test} test\n'
            f'Using {dataloader.num_workers} dataloader workers\n'
            f'Logging results to {save_dir}\n'
            f'Starting training for {epochs} epochs...')
# torch.save(model, wdir / 'init.pt')

Image sizes 1280 train, 1280 test
Using 0 dataloader workers
Logging results to runs/train/exp
Starting training for 10 epochs...


In [25]:
for epoch in range(start_epoch, epochs):  # epoch ------------------------------------------------------------------
    with torch.no_grad():
        model.train()
        print(epoch)
        # Update image weights (optional)
        if opt.image_weights:
            # Generate indices
            if rank in [-1, 0]:
                cw = model.class_weights.cpu().numpy() * (1 - maps) ** 2 / nc  # class weights
                iw = labels_to_image_weights(dataset.labels, nc=nc, class_weights=cw)  # image weights
                dataset.indices = random.choices(range(dataset.n), weights=iw, k=dataset.n)  # rand weighted idx
            # Broadcast if DDP
            if rank != -1:
                indices = (torch.tensor(dataset.indices) if rank == 0 else torch.zeros(dataset.n)).int()
                dist.broadcast(indices, 0)
                if rank != 0:
                    dataset.indices = indices.cpu().numpy()

        # Update mosaic border
        # b = int(random.uniform(0.25 * imgsz, 0.75 * imgsz + gs) // gs * gs)
        # dataset.mosaic_border = [b - imgsz, -b]  # height, width borders

        mloss = torch.zeros(4, device=device)  # mean losses
        if rank != -1:
            dataloader.sampler.set_epoch(epoch)
        pbar = enumerate(dataloader)
        logger.info(('\n' + '%10s' * 8) % ('Epoch', 'gpu_mem', 'box', 'obj', 'cls', 'total', 'labels', 'img_size'))
        if rank in [-1, 0]:
            pbar = tqdm(pbar, total=nb)  # progress bar
        optimizer.zero_grad()
        for i, (imgs, targets, paths, _) in pbar:  # batch -------------------------------------------------------------
            ni = i + nb * epoch  # number integrated batches (since train start)
            imgs = imgs.to(device, non_blocking=True).float() / 255.0  # uint8 to float32, 0-255 to 0.0-1.0

            # Warmup
            if ni <= nw:
                xi = [0, nw]  # x interp
                # model.gr = np.interp(ni, xi, [0.0, 1.0])  # iou loss ratio (obj_loss = 1.0 or iou)
                accumulate = max(1, np.interp(ni, xi, [1, nbs / total_batch_size]).round())
                for j, x in enumerate(optimizer.param_groups):
                    # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                    x['lr'] = np.interp(ni, xi, [hyp['warmup_bias_lr'] if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
                    if 'momentum' in x:
                        x['momentum'] = np.interp(ni, xi, [hyp['warmup_momentum'], hyp['momentum']])

            # Multi-scale
            if opt.multi_scale:
                sz = random.randrange(imgsz * 0.5, imgsz * 1.5 + gs) // gs * gs  # size
                sf = sz / max(imgs.shape[2:])  # scale factor
                if sf != 1:
                    ns = [math.ceil(x * sf / gs) * gs for x in imgs.shape[2:]]  # new shape (stretched to gs-multiple)
                    imgs = F.interpolate(imgs, size=ns, mode='bilinear', align_corners=False)
            # print(imgs)
            print(targets.shape)
            # Forward
            with amp.autocast(enabled=cuda):
                pred = model(imgs)  # forward
                loss, loss_items = compute_loss_ota(pred, targets.to(device), imgs)
                # print(pred)
                # torch.save(pred,'predict_sample.pt')
                # torch.save(targets, 'target_sample.pt')
                # torch.save(imgs, 'imgs_sample.pt')
                # loss, loss_items = compute_loss(pred, targets.to(device))
                print(loss)
            break
        
        # if rank in [-1, 0]:
        #         # mAP
        #         ema.update_attr(model, include=['yaml', 'nc', 'hyp', 'gr', 'names', 'stride', 'class_weights'])
        #         final_epoch = epoch + 1 == epochs
        #         if not opt.notest or final_epoch:  # Calculate mAP
        #             wandb_logger.current_epoch = epoch + 1
        #             results, maps, times = test.test(data_dict,
        #                                             batch_size=batch_size * 2,
        #                                             imgsz=imgsz_test,
        #                                             model=ema.ema,
        #                                             single_cls=opt.single_cls,
        #                                             dataloader=testloader,
        #                                             save_dir=save_dir,
        #                                             verbose=nc < 50 and final_epoch,
        #                                             plots=plots and final_epoch,
        #                                             wandb_logger=wandb_logger,
        #                                             compute_loss=compute_loss,
        #                                             is_coco=is_coco,
        #                                             v5_metric=opt.v5_metric)
        
        break



     Epoch   gpu_mem       box       obj       cls     total    labels  img_size


0


  0%|          | 0/36498 [00:00<?, ?it/s]

torch.Size([4, 234])


  0%|          | 0/36498 [00:00<?, ?it/s]

tensor([0.59060], device='cuda:0')


In [26]:
import torch

a = torch.tensor([[0,1],
                  [0,2],
                  [0,3]])

In [27]:
a

tensor([[0, 1],
        [0, 2],
        [0, 3]])

In [28]:
a[0]

tensor([0, 1])

In [30]:
torch.where(a[0] == 1)[0]

tensor([1])